In [3]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

meas = ClassicalRegister(1, 'cr0')
_ansilla = QuantumRegister(1, 'ansl')
qi = QuantumRegister(2, 'qr')
circuit = QuantumCircuit(qi, _ansilla, meas)

theta = np.random.uniform(0, 2*np.pi, size=4)
phi = np.random.uniform(0, 2*np.pi, size=4)

# Ui
ui = QuantumCircuit(qi)
ui.h(qi)
ui.x(qi[0])
ui.crx(theta[1]-theta[0], qi[0], qi[1], label='R(t1)')
ui.x(qi)
ui.crx(theta[2]-theta[0], qi[0], qi[1], label='R(t2)')
ui.x(qi[1])
ui.crx(theta[3]-theta[0], qi[0], qi[1], label='R(t3)')
print('Ui:')
print(ui.draw())
uiGate = ui.to_gate(label='Ui')

circuit.append(uiGate, qi)
circuit.barrier()

# Uw
uw = QuantumCircuit(qi)
uw.x(qi[0])
uw.crx(phi[1]-phi[0], qi[0], qi[1], label='R(p1)')
uw.x(qi)
uw.crx(phi[2]-phi[0], qi[0], qi[1], label='R(p2)')
uw.x(qi[1])
uw.crx(phi[3]-phi[0], qi[0], qi[1], label='R(p3)')
uw.h(qi)
uw.x(qi)
print('Uw:')
print(uw.draw())

uwGate = uw.to_gate(label='Uw')
circuit.append(uwGate, qi)
circuit.barrier()

circuit.ccx(qi[0], qi[1], _ansilla)
circuit.measure(_ansilla, meas)

circuit.draw()

Ui:
      ┌───┐┌───┐    R(t1)     ┌───┐    R(t2)               R(t3)      
qr_0: ┤ H ├┤ X ├──────■───────┤ X ├──────■───────────────────■────────
      ├───┤└───┘┌─────┴──────┐├───┤┌─────┴──────┐┌───┐┌──────┴───────┐
qr_1: ┤ H ├─────┤ Rx(3.0296) ├┤ X ├┤ Rx(2.7239) ├┤ X ├┤ Rx(-0.92325) ├
      └───┘     └────────────┘└───┘└────────────┘└───┘└──────────────┘
Uw:
      ┌───┐    R(p1)     ┌───┐    R(p2)              R(p3)      ┌───┐┌───┐
qr_0: ┤ X ├──────■───────┤ X ├──────■──────────────────■────────┤ H ├┤ X ├
      └───┘┌─────┴──────┐├───┤┌─────┴─────┐┌───┐┌──────┴───────┐├───┤├───┤
qr_1: ─────┤ Rx(1.0935) ├┤ X ├┤ Rx(1.073) ├┤ X ├┤ Rx(-0.82824) ├┤ H ├┤ X ├
           └────────────┘└───┘└───────────┘└───┘└──────────────┘└───┘└───┘


┌─────┐ ░ ┌─────┐ ░         
 qr_0: ┤0    ├─░─┤0    ├─░───■─────
       │  Ui │ ░ │  Uw │ ░   │     
 qr_1: ┤1    ├─░─┤1    ├─░───■─────
       └─────┘ ░ └─────┘ ░ ┌─┴─┐┌─┐
 ansl: ────────░─────────░─┤ X ├┤M├
               ░         ░ └───┘└╥┘
cr0: 1/══════════════════════════╩═
                                 0

In [28]:
from qiskit.quantum_info.operators import Operator

def nBin(n, x):
    return (''.join('0' for _ in range(n)) + bin(x)[2:])[-n:]

def Rcontrol(n, th):
    circ = QuantumCircuit(1)
    r = Operator([
        [1, 0],
        [0, np.exp(np.imag(th))]
    ])
    circ.unitary(r, [0])
    gate = circ.to_gate(label='R(' + str(np.round(th, 2)) + ')')
    rc = gate.control(n-1)
    return rc

def nToffoli(n):
    x_circ = QuantumCircuit(1, name='X')
    x_circ.x(0)
    x_gate = x_circ.to_gate()
    cx_gate = x_gate.control(n)
    return cx_gate

def Ui(n, theta):
    q = QuantumRegister(n, 'qr')
    ui = QuantumCircuit(q)
    ui.h(q)
    
    for s in range(1, np.power(2, n)):
        step = nBin(n, s)
        apply_x = np.zeros(n)
        for i, pos in enumerate(step):
            if pos == '0':
                ui.x(q[i])
                apply_x[i] = 1

        th = theta[s] - theta[0]
        rc = Rcontrol(n, th)
        ui.append(rc, [q[i] for i in range(n)])

        for p, x in enumerate(apply_x):
            if x == 1:
                ui.x(q[p])
    return ui

def Uw(n, phi):
    q = QuantumRegister(n, 'qr')
    uw = QuantumCircuit(q)
    
    for s in range(1, np.power(2, n)):
        step = nBin(n, s)
        apply_x = np.zeros(n)
        for i, pos in enumerate(step):
            if pos == '0':
                uw.x(q[i])
                apply_x[i] = 1

        th = phi[s] - phi[0]
        rc = Rcontrol(n, th)
        uw.append(rc, [q[i] for i in range(n)])

        for p, x in enumerate(apply_x):
            if x == 1:
                uw.x(q[p])
    
    uw.h(q)
    uw.x(q)

    return uw
    

theta = np.random.uniform(0, 2*np.pi, size=8)
phi = np.random.uniform(0, 2*np.pi, size=8)
Ui(3, theta).draw()
Uw(3, phi).draw()

┌───┐            ┌───┐┌───┐           ┌───┐┌───┐           ┌───┐»
qr_0: ┤ X ├─────■──────┤ X ├┤ X ├─────■─────┤ X ├┤ X ├─────■─────┤ X ├»
      ├───┤     │      ├───┤└───┘     │     └───┘└───┘     │     ├───┤»
qr_1: ┤ X ├─────■──────┤ X ├──────────■────────────────────■─────┤ X ├»
      └───┘┌────┴─────┐├───┤     ┌────┴────┐┌───┐     ┌────┴────┐├───┤»
qr_2: ─────┤ R(-0.34) ├┤ X ├─────┤ R(4.17) ├┤ X ├─────┤ R(3.62) ├┤ X ├»
           └──────────┘└───┘     └─────────┘└───┘     └─────────┘└───┘»
«                                                                      ┌───┐»
«qr_0: ─────■───────────────────■───────────────■────────────────■─────┤ H ├»
«           │     ┌───┐┌───┐    │     ┌───┐     │                │     ├───┤»
«qr_1: ─────■─────┤ X ├┤ X ├────■─────┤ X ├─────■────────────────■─────┤ H ├»
«      ┌────┴────┐├───┤└───┘┌───┴────┐├───┤┌────┴─────┐┌───┐┌────┴────┐├───┤»
«qr_2: ┤ R(2.14) ├┤ X ├─────┤ R(0.2) ├┤ X ├┤ R(-0.28) ├┤ X ├┤ R(2.76) ├┤ H ├»
«      └─────────┘└───┘     └────────┘└───┘└──────────┘└───┘└─────────┘└───┘»
«      ┌───┐
«qr_0: ┤ X ├
«      ├───┤
«qr_1: ┤ X ├
«      ├───┤
«qr_2: ┤ X ├
«      └───┘

In [31]:
def perceptron(n, theta, phi):
    qr = QuantumRegister(n, 'qr')
    ansilla = QuantumRegister(1, 'ansl')
    meas = ClassicalRegister(1, 'cr0')
    circuit = QuantumCircuit(qr, ansilla, meas)

    ui = Ui(n, theta).to_gate(label='Ui')
    uw = Uw(n, phi).to_gate(label='Uw')

    circuit.append(ui, qr)
    circuit.barrier()
    circuit.append(uw, qr)
    circuit.barrier()
    
    ntf = nToffoli(n)
    qargs = [qr[i] for i in range(n)] + [ansilla]
    circuit.append(ntf, qargs)
    
    circuit.measure(ansilla, meas)
    return circuit

theta = np.random.uniform(0, 2*np.pi, size=8)
phi = np.random.uniform(0, 2*np.pi, size=8)

perceptron(3, theta, phi).draw()

┌─────┐ ░ ┌─────┐ ░         
 qr_0: ┤0    ├─░─┤0    ├─░───■─────
       │     │ ░ │     │ ░   │     
 qr_1: ┤1 Ui ├─░─┤1 Uw ├─░───■─────
       │     │ ░ │     │ ░   │     
 qr_2: ┤2    ├─░─┤2    ├─░───■─────
       └─────┘ ░ └─────┘ ░ ┌─┴─┐┌─┐
 ansl: ────────░─────────░─┤ X ├┤M├
               ░         ░ └───┘└╥┘
cr0: 1/══════════════════════════╩═
                                 0

In [1]:
import numpy as np
from perceptron import QPerceptron

n = 3
theta = np.random.uniform(0, 2*np.pi, size=8)
phi = np.random.uniform(0, 2*np.pi, size=8)

p = QPerceptron(n, theta, phi)
p.draw_uw()

      ┌───┐                     ┌───┐                    ┌───┐»
qr_0: ┤ X ├─────────────────────┤ X ├─────────■──────────┤ X ├»
      ├───┤                     └─┬─┘         │          └─┬─┘»
qr_1: ┤ X ├──────────■────────────■───────────┼────────────■──»
      └───┘┌─────────┴─────────┐     ┌────────┴─────────┐     »
qr_2: ─────┤ U(0,0,-0.84402,0) ├─────┤ U(0,0.84402,0,0) ├─────»
           └───────────────────┘     └──────────────────┘     »
«                            ░ ┌───┐        ┌───┐        ┌───┐»
«qr_0: ──────────■───────────░─┤ X ├────────┤ X ├────────┤ X ├»
«                │           ░ ├───┤        └───┘        └─┬─┘»
«qr_1: ──────────┼───────────░─┤ X ├──────────■────────────■──»
«      ┌─────────┴─────────┐ ░ ├───┤┌─────────┴─────────┐     »
«qr_2: ┤ U(0,0,-0.84402,0) ├─░─┤ X ├┤ U(0,0,-0.96445,0) ├─────»
«      └───────────────────┘ ░ └───┘└───────────────────┘     »
«                          ┌───┐                      ░ ┌───┐»
«qr_0: ─────────■──────────┤ X ├─────────